In [1]:
import pandas as pd
import json
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM
from concurrent.futures import ThreadPoolExecutor
import torch


# def chat(user_input):
#     model_path = "../exp_model/v1"
#     tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
#     model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype="auto", device_map="auto", trust_remote_code=True)
#     messages = [
#         {"role": "system", "content": ''},
#         {"role": "user", "content": user_input}
#     ]

#     input_ids = tokenizer.apply_chat_template(
#         messages, add_generation_prompt=True, return_tensors="pt"
#     ).to(model.device)

#     outputs = model.generate(
#         input_ids,
#         max_new_tokens=1024,
#         do_sample=True,
#         temperature=0.5,
#         top_p=0.5,
#     )
#     response = outputs[0][input_ids.shape[-1]:]
#     return tokenizer.decode(response, skip_special_tokens=True)

model_path = "../exp_model/v1"
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype="auto", device_map="auto", trust_remote_code=True)


def process_row(row):
    try:
        stock_mapping = {
            "600031.SH": "三一重工",
            "600036.SH": "招商银行",
            "600050.SH": "中国联通",
            "600104.SH": "上汽集团",
            "600346.SH": "恒力石化",
            "600570.SH": "恒生电子",
            "600887.SH": "伊利股份",
            "601390.SH": "中国中铁",
            "603160.SH": "汇项科技",
            "601668.SH": "中国建筑"
        }
        stock = stock_mapping[row['symbol']]
        prompt = f"请根据以下新闻文本，预测{stock}股票的对数收益率属于以下哪一类别 (极度负面/负面/中性/正面/极度正面)"
        label_mapping = {0: "极度负面", 1: "负面", 2: "中性", 3: "正面", 4: "极度正面"}
        label = label_mapping[row['label']]

        def chat(user_input):
            messages = [
                {"role": "system", "content": ''},
                {"role": "user", "content": user_input}
            ]

            input_ids = tokenizer.apply_chat_template(
                messages, add_generation_prompt=True, return_tensors="pt"
            ).to(model.device)

            outputs = model.generate(
                input_ids,
                max_new_tokens=1024,
                do_sample=True,
                temperature=0.5,
                top_p=0.5,
            )
            response = outputs[0][input_ids.shape[-1]:]
            return tokenizer.decode(response, skip_special_tokens=True)

        predict = chat(prompt+row['content'])
        torch.cuda.empty_cache()
        return {
            "prompt": prompt,
            "content": row['content'],
            "label": label,
            "predict": predict
        }
    except Exception as e:
        print(f"Error processing row: {e}")
        return None


def generate_dataset(df, max_samples=32):
    df_subset = df.head(max_samples)
    results = []
    with ThreadPoolExecutor(max_workers=2) as executor:
        futures = [executor.submit(process_row, row) for _, row in df_subset.iterrows()]
        for future in tqdm(futures, total=len(futures), desc="Processing rows"):
            result = future.result()
            if result:
                results.append(result)

    return results

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [2]:
data = pd.read_parquet('processed_LLM_data_test_1_3_30.parquet')
data

,symbol,time,content,log_r
489,600031.SH,2023-04-14 09:00:00,现在是2023-04-14 09:00:00+08:00\n时间：2023-04-09 17...,-0.015666
490,600031.SH,2023-04-25 09:00:00,现在是2023-04-25 09:00:00+08:00\n时间：2023-04-10 18...,-0.031186
491,600031.SH,2023-04-25 09:00:00,现在是2023-04-25 09:00:00+08:00\n时间：2023-04-13 23...,-0.031186
492,600031.SH,2023-04-26 09:00:00,现在是2023-04-26 09:00:00+08:00\n时间：2023-04-24 09...,0.005575
493,600031.SH,2023-04-26 09:00:00,现在是2023-04-26 09:00:00+08:00\n时间：2023-04-24 09...,0.005575
...,...,...,...,...
6482,601668.SH,2024-03-21 09:00:00,现在是2024-03-21 09:00:00+08:00\n时间：2024-03-20 19...,0.005764
6483,601668.SH,2024-03-21 09:00:00,现在是2024-03-21 09:00:00+08:00\n时间：2024-03-20 19...,0.005764
6484,601668.SH,2024-03-25 09:00:00,现在是2024-03-25 09:00:00+08:00\n时间：2024-03-20 19...,-0.007707
6485,601668.SH,2024-03-29 09:00:00,现在是2024-03-29 09:00:00+08:00\n时间：2024-03-20 19...,0.001914


In [3]:
data['label'] = pd.qcut(data['log_r'], 5, labels=False)
print(data[['log_r', 'label']].head())

        log_r  label
489 -0.015666      0
490 -0.031186      0
491 -0.031186      0
492  0.005575      3
493  0.005575      3


In [4]:
processed_data = generate_dataset(data, max_samples=10000)
output_file_path = 'ft_data_score_result.json'
with open(output_file_path, 'w', encoding='utf-8') as f:
    json.dump(processed_data, f, ensure_ascii=False, indent=2)

Processing rows:   0%|          | 3/1400 [00:01<06:35,  3.53it/s]

Error processing row: CUDA out of memory. Tried to allocate 648.00 MiB. GPU 0 has a total capacity of 23.69 GiB of which 405.12 MiB is free. Process 2068527 has 440.00 MiB memory in use. Process 2357580 has 1.17 GiB memory in use. Process 2745936 has 18.10 GiB memory in use. Including non-PyTorch memory, this process has 3.57 GiB memory in use. Of the allocated memory 3.27 GiB is allocated by PyTorch, and 6.45 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)
Error processing row: CUDA out of memory. Tried to allocate 1.16 GiB. GPU 0 has a total capacity of 23.69 GiB of which 253.12 MiB is free. Process 2068527 has 440.00 MiB memory in use. Process 2357580 has 1.17 GiB memory in use. Process 2745936 has 18.10 GiB memory in use. Including non-PyTorch mem

Processing rows:   0%|          | 4/1400 [00:01<05:35,  4.16it/s]

Error processing row: CUDA out of memory. Tried to allocate 344.00 MiB. GPU 0 has a total capacity of 23.69 GiB of which 135.12 MiB is free. Process 2068527 has 440.00 MiB memory in use. Process 2357580 has 1.17 GiB memory in use. Process 2745936 has 18.10 GiB memory in use. Including non-PyTorch memory, this process has 3.83 GiB memory in use. Of the allocated memory 2.92 GiB is allocated by PyTorch, and 627.63 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)
Error processing row: CUDA out of memory. Tried to allocate 144.00 MiB. GPU 0 has a total capacity of 23.69 GiB of which 57.12 MiB is free. Process 2068527 has 440.00 MiB memory in use. Process 2357580 has 1.17 GiB memory in use. Process 2745936 has 18.10 GiB memory in use. Including non-PyTorch 

Processing rows:   1%|          | 8/1400 [00:01<02:46,  8.35it/s]

...

Error processing row: CUDA out of memory. Tried to allocate 2.65 GiB. GPU 0 has a total capacity of 23.69 GiB of which 2.58 GiB is free. Process 2068527 has 440.00 MiB memory in use. Process 2357580 has 1.17 GiB memory in use. Process 2745936 has 2.99 GiB memory in use. Including non-PyTorch memory, this process has 16.49 GiB memory in use. Of the allocated memory 13.89 GiB is allocated by PyTorch, and 2.28 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)
Error processing row: The expanded size of the tensor (35515) must match the existing size (32767) at non-singleton dimension 1.  Target sizes: [1, 35515, 32, 128].  Tensor sizes: [1, 32767, 1, 1]
Error processing row: The expanded size of the tensor (48545) must match the existing size (32767) at non

Processing rows:  34%|███▍      | 473/1400 [05:37<29:24,  1.90s/it]

Error processing row: CUDA out of memory. Tried to allocate 420.00 MiB. GPU 1 has a total capacity of 23.69 GiB of which 285.00 MiB is free. Process 2745936 has 5.66 GiB memory in use. Including non-PyTorch memory, this process has 17.73 GiB memory in use. Of the allocated memory 14.66 GiB is allocated by PyTorch, and 2.76 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)
Error processing row: CUDA out of memory. Tried to allocate 122.00 MiB. GPU 1 has a total capacity of 23.69 GiB of which 41.00 MiB is free. Process 2745936 has 5.66 GiB memory in use. Including non-PyTorch memory, this process has 17.97 GiB memory in use. Of the allocated memory 15.02 GiB is allocated by PyTorch, and 2.64 GiB is reserved by PyTorch but unallocated. If reserved but unal

Processing rows:  34%|███▍      | 474/1400 [05:42<11:08,  1.39it/s]


Error processing row: probability tensor contains either `inf`, `nan` or element < 0
Error processing row: probability tensor contains either `inf`, `nan` or element < 0
Error processing row: CUDA out of memory. Tried to allocate 448.00 MiB. GPU 1 has a total capacity of 23.69 GiB of which 397.00 MiB is free. Process 2745936 has 5.66 GiB memory in use. Including non-PyTorch memory, this process has 17.62 GiB memory in use. Of the allocated memory 14.60 GiB is allocated by PyTorch, and 2.71 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)
Error processing row: The expanded size of the tensor (35090) must match the existing size (32767) at non-singleton dimension 1.  Target sizes: [1, 35090, 32, 128].  Tensor sizes: [1, 32767, 1, 1]
Error processing row: